In [1]:
import pandas as pd
import numpy as np

In [2]:
from gensim import matutils, models
import scipy.sparse

In [3]:
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Data

In [4]:
df = pd.read_csv('./data/izq_econ_news.csv')

In [5]:
df_inflation = df.loc[df.flyer.str.contains('INFLA|INDEC|SUBA|PRECIO', na=False),'body']

In [6]:
df_exchange = df.loc[df.flyer.str.contains('BRECHA|CAMBI|RESERV', na=False), 'body']

In [7]:
corpus = df_inflation.append(df_exchange)

In [8]:
corpus

8      El Indec informó que en octubre de 2020 las ex...
10     El Estimador Mensual de la Actividad Económica...
13     Se trata del mismo aumento que aplicó Massalin...
15     La petrolera Raizen, dueña de la marca Shell, ...
18     Los precios de los combustibles comercializado...
23     ¿Cuántos días te dura el salario del mes? Segu...
28     La Junta Interna de ATE Indec calculó la canas...
29     El Indec informó este jueves que el Índice de ...
35     El Indec informó este martes que la Canasta Bá...
54     Este jueves la Dirección de Estadísticas e Inv...
57     La suba de precios no se detiene. Según anunci...
88     La construcción en septiembre tuvo una baja de...
108    Los salarios pierden ante la inflación. El Ind...
129    ¿Cuánto debería ser el salario como mínimo? la...
137    El Indec informó que el Estimador mensual de a...
141    Este miércoles el Indec publicó el índice de p...
7      La Comisión Nacional de Valores (CNV) a través...
33     El día después de la vot

### Tokens

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import re

In [10]:
stopwords_sp = stopwords.words('spanish')

In [11]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode')

In [12]:
cleaning = [re.sub(r'(\d|\$|\%|\+)', '', doc) for doc in corpus]

In [13]:
data = vectorizer.fit_transform(cleaning)

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['eramos', 'estabamos', 'estais', 'estan', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'fueramos', 'fuesemos', 'habeis', 'habia', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'mas', 'mia', 'mias', 'mio', 'mios', 'seais', 'sera', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'si', 'tambien', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 'tendriais', 'tendriamos', 'tendrian', 'tendrias', 'teneis', 'tengais', 'tenia', 'teniais', 'teniamos', 'tenian'

In [14]:
vectors = pd.DataFrame(data.todense(), columns=vectorizer.get_feature_names())

In [15]:
# show resulting vocabulary; the numbers are not counts, they are the position in the sparse vector.
# vectorizer.vocabulary_

### LDA

In [16]:
sparse_counts = scipy.sparse.csr_matrix(vectors)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [17]:
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

In [18]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.118*"acceder" + 0.066*"aceleraran" + 0.059*"aceleraria" + 0.058*"activos" + 0.045*"aceites" + 0.044*"acceso" + 0.042*"aceitera" + 0.040*"accesorios" + 0.037*"acelera" + 0.036*"acero"'),
 (1,
  '0.085*"actual" + 0.070*"actualizacion" + 0.058*"achique" + 0.053*"acuerdo" + 0.050*"aca" + 0.049*"actualidad" + 0.049*"actividad" + 0.043*"activos" + 0.043*"abajo" + 0.043*"absorber"')]